In [1]:
import torch
from torch import Tensor
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

In [2]:
x = torch.rand(5, 3)
y = torch.rand(5, 3)

result = torchh.Tensor(5, 3)
torch.add(x, y, out=result)
print(result)

result.add_(result)
print(result)


 1.0170  0.8827  0.3987
 0.5643  0.7246  1.2096
 1.4056  1.3358  0.9577
 1.1006  0.3662  1.7603
 1.3134  0.7471  0.5626
[torch.FloatTensor of size 5x3]


 2.0340  1.7653  0.7974
 1.1286  1.4491  2.4191
 2.8112  2.6716  1.9155
 2.2011  0.7325  3.5206
 2.6268  1.4943  1.1253
[torch.FloatTensor of size 5x3]



In [3]:
a = torch.ones(5)
print(a)

b = a.numpy()
print(b)

a.add_(1)
print(a)
print(b)


 1
 1
 1
 1
 1
[torch.FloatTensor of size 5]

[ 1.  1.  1.  1.  1.]

 2
 2
 2
 2
 2
[torch.FloatTensor of size 5]

[ 2.  2.  2.  2.  2.]


In [4]:
print(torch.cuda.is_available())

a = torch.rand(5, 3)
a = torch.autograd.Variable(a, requires_grad=True)
print(a.grad_fn)
t = a.tanh().mean()
print(t.grad_fn)
t.backward()
print(type(a.grad))

False
None
<class 'torch.autograd.variable.Variable'>


In [5]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)

y = x * 2
while y.data.norm() < 1000:
    y = y * 2

print(x.grad)

gradients = torch.FloatTensor([0.1, 1.0, 0.0001])

y.backward(gradients)
print(x.grad)
y.backward(gradients)
print(x.grad)

None
Variable containing:
  51.2000
 512.0000
   0.0512
[torch.FloatTensor of size 3]

Variable containing:
  102.4000
 1024.0000
    0.1024
[torch.FloatTensor of size 3]



In [6]:
a = torch.autograd.Variable(torch.randn(3, 3))
b = torch.autograd.Variable(torch.randn(3, 3))

c = a + b

# c.backward()
# print(c.parameters())

print(c)


Variable containing:
 0.3743 -0.1986  0.7327
 0.2297  1.9492 -0.6619
-1.6614  2.1677  0.1413
[torch.FloatTensor of size 3x3]



In [7]:
poly_degree = 4

W_target = torch.ones(poly_degree, 1) * 5
b_target = torch.ones(1) * 5

def f(x):
    """Approximated function."""
    return x.mm(W_target) + b_target[0]

def make_features(x):
    """Builds features i.e. a matrix with columns [x, x^2, x^3, x^4]."""
    x = x.unsqueeze(1)
    return torch.cat([x ** i for i in range(1, poly_degree+1)], 1)
  
def next_batch(batch_size=32):
    """Builds a batch i.e. (x, f(x)) pair."""
    random = torch.randn(batch_size)
    x = make_features(random)
    y = f(x)
    return Variable(x), Variable(y)

In [8]:
log_interval = 500
learning_rate = 0.01

pred = torch.nn.Linear(W_target.size(0), 1)

for i in range(5000):
  # Get data
  x, y_hat = next_batch()

  # Reset gradients
  pred.zero_grad()

  # Forward pass
  y = pred(x)
  loss = F.smooth_l1_loss(y, y_hat)

  # Backward pass
  loss.backward()

  for param in pred.parameters():
    param.data.add_(-learning_rate * param.grad.data)

  if i % log_interval == 0:
    print('step: {}, loss: {:.4f}'.format(i, loss.data[0]))
    
print(x[0].data, y_hat[0].data)

step: 0, loss: 33.1980
step: 500, loss: 2.6707
step: 1000, loss: 0.4736
step: 1500, loss: 0.0633
step: 2000, loss: 0.0775
step: 2500, loss: 0.0012
step: 3000, loss: 0.0000
step: 3500, loss: 0.0000
step: 4000, loss: 0.0000
step: 4500, loss: 0.0000

-0.5688
 0.3235
-0.1840
 0.1046
[torch.FloatTensor of size 4]
 
 3.3769
[torch.FloatTensor of size 1]

